![NYC Skyline](img/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many [Airbnb](https://www.airbnb.com/) listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this notebook, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

- What is the average price, per night, of an Airbnb listing in NYC?
- How does the average price of an Airbnb listing, per month, compare to the private rental market?
- How many adverts are for private rooms?
- How do Airbnb listing prices compare across the five NYC boroughs?


In [331]:
# We've loaded your first package for you! You can add as many cells as you need.
import numpy as np
import pandas as pd
import datetime as dt

In [332]:
#Importing the data

prices = pd.read_csv("data/airbnb_price.csv")
xls = pd.ExcelFile("data/airbnb_room_type.xlsx")
room_types = xls.parse(0)
reviews = pd.read_csv("data/airbnb_last_review.tsv", "\t")

In [333]:
#Cleaning the price column
print(prices)
prices['price'] = prices['price'].str.replace('dollars', '')
prices.price.str.strip()
prices["price"] = pd.to_numeric(prices["price"])
print(prices)

       listing_id        price                 nbhood_full
0            2595  225 dollars          Manhattan, Midtown
1            3831   89 dollars      Brooklyn, Clinton Hill
2            5099  200 dollars      Manhattan, Murray Hill
3            5178   79 dollars   Manhattan, Hell's Kitchen
4            5238  150 dollars        Manhattan, Chinatown
...           ...          ...                         ...
25204    36425863  129 dollars  Manhattan, Upper East Side
25205    36427429   45 dollars            Queens, Flushing
25206    36438336  235 dollars  Staten Island, Great Kills
25207    36442252  100 dollars           Bronx, Mott Haven
25208    36455809   30 dollars          Brooklyn, Bushwick

[25209 rows x 3 columns]
       listing_id  price                 nbhood_full
0            2595    225          Manhattan, Midtown
1            3831     89      Brooklyn, Clinton Hill
2            5099    200      Manhattan, Murray Hill
3            5178     79   Manhattan, Hell's Kitchen
4

In [334]:
#Calculating average price

#prices['price'].describe()

free_listings = prices["price"] == 0

prices = prices.loc[~free_listings]


avg_price = round(prices["price"].mean(), 2)

print("The average price of rent is £" + str(avg_price))

prices.head()


The average price of rent is £141.82


,listing_id,price,nbhood_full
0,2595,225,"Manhattan, Midtown"
1,3831,89,"Brooklyn, Clinton Hill"
2,5099,200,"Manhattan, Murray Hill"
3,5178,79,"Manhattan, Hell's Kitchen"
4,5238,150,"Manhattan, Chinatown"


In [335]:
#Comparing costs to the private rental market

prices['price_per_month'] = round(prices['price'] * 365 / 12 ,2)
average_price_per_month = round(prices['price_per_month'].mean(),2)
difference = round((average_price_per_month - 3100),2)
prices.head()

,listing_id,price,nbhood_full,price_per_month
0,2595,225,"Manhattan, Midtown",6843.75
1,3831,89,"Brooklyn, Clinton Hill",2707.08
2,5099,200,"Manhattan, Murray Hill",6083.33
3,5178,79,"Manhattan, Hell's Kitchen",2402.92
4,5238,150,"Manhattan, Chinatown",4562.50


In [336]:
#Cleaning the room_type column

room_types['room_type'] = room_types['room_type'].str.lower()

room_types["room_type"] = room_types["room_type"].astype("category")

room_frequencies = room_types["room_type"].value_counts()


In [337]:
#What timeframe are we working with?

reviews["last_review"]  = pd.to_datetime(reviews["last_review"])

first_reviewed = reviews["last_review"].dt.date.min()
last_reviewed = reviews["last_review"].dt.date.max()


In [338]:
#Joining the dataframe

rooms_and_prices = prices.merge(room_types,how='outer',on="listing_id")
airbnb_merged = rooms_and_prices.merge(reviews,how='outer',on="listing_id")
airbnb_merged.dropna(inplace=True)


In [339]:
#Analzing listing prices by NYC borough
airbnb_merged.head()
airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(",")[0]

boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])
boroughs = boroughs.round(2).sort_values("mean", ascending=False)
boroughs.head()

,sum,mean,median,count
borough,,,,
Manhattan,1898417.0,184.04,149.0,10315
Brooklyn,1275250.0,122.02,95.0,10451
Queens,320715.0,92.83,70.0,3455
Staten Island,22974.0,86.04,71.0,267
Bronx,55156.0,79.25,65.0,696


In [340]:
#Price range by borough
label_names=["Budget","Average","Expensive","Extravagant"]
ranges = [0,69,175,350, np.inf]
airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)
prices_by_borough = airbnb_merged.groupby(["borough", "price_range"])["price_range"].count()

In [341]:
#Storing the final result

solution = {'avg_price':avg_price,
            'average_price_per_month': average_price_per_month,  
            'difference':difference,          
            'room_frequencies':room_frequencies, 
            'first_reviewed': first_reviewed,
            'last_reviewed': last_reviewed,
            'prices_by_borough':prices_by_borough}
print(solution)

{'avg_price': 141.82, 'average_price_per_month': 4313.61, 'difference': 1213.61, 'room_frequencies': entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64, 'first_reviewed': datetime.date(2019, 1, 1), 'last_reviewed': datetime.date(2019, 7, 9), 'prices_by_borough': borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive  